# RFM Analysis - Pension Contributions Demo

This iPython notebook explains how to perform RFM analysis from customer pension contribution history data. The sample pension contribution orders file is a sample dataset.

If you have suggestions or improvements please contribute on https://github.com/caiomsouza/rfm-pension-contributions-demo

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [9]:
pensions_contributions = pd.read_csv('sample-pensions-contributions.csv',sep=',')

In [32]:
pensions_contributions.head()

,pension_contribution_date,pension_contribution_id,member_name,contribution_amount
0,2012-01-13,CA-2012-137526,Philip Brown,612
1,2012-01-13,CA-2012-169677,Karen Seio,10
2,2014-01-13,CA-2014-108000,Linda Southworth,10
3,2014-01-13,US-2014-148551,David Bremer,761
4,2014-01-13,US-2014-158512,Dianna Arnett,78


# Data Dictionary

<b>pension_contribution_date</b> = Date when the member made a contribution to his/her pension <BR>
<b>pension_contribution_id</b> = Contribution ID <BR>
<b>member_name</b> = Name of the Member who made the contribution <BR>
<b>contribution_amount</b> = The total amount of the member pension contribution  <BR>




## Create the RFM Table

Since recency is calculated for a point in time and the Tableau Super Store dataset last order date is Dec 31 2014, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [11]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [12]:
# Make the date_placed column datetime
pensions_contributions['pension_contribution_date'] = pd.to_datetime(pensions_contributions['pension_contribution_date'])

Create the RFM Table

In [13]:
rfmTable = pensions_contributions.groupby('member_name').agg({'pension_contribution_date': lambda x: (NOW - x.max()).days, # Recency
                                        'pension_contribution_id': lambda x: len(x),      # Frequency
                                        'contribution_amount': lambda x: x.sum()}) # Monetary Value

rfmTable['pension_contribution_date'] = rfmTable['pension_contribution_date'].astype(int)
rfmTable.rename(columns={'pension_contribution_date': 'recency', 
                         'pension_contribution_id': 'frequency', 
                         'contribution_amount': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [15]:
rfmTable.head()

,recency,frequency,monetary_value
member_name,,,
Amy Hunt,1079,1,89
Andy Reiter,348,1,5803
Anna Gayman,716,1,405
Bill Eplett,715,1,82
Brendan Sweed,1446,1,62


Recency = Number of days since the last pension contribution <BR>
Frequency = Number of times the member made a pension contribution <BR>
Monetary Value = Total Amount contributed by the member <BR>

Customer **Aaron Bergman** has frequency:3, monetary value:$887 and recency:415 days.

In [18]:
amy_hunt = pensions_contributions[pensions_contributions['member_name']=='Amy Hunt']
amy_hunt

,pension_contribution_date,pension_contribution_id,member_name,contribution_amount
28,2012-01-17,CA-2012-151547,Amy Hunt,89


Inserting the date of Aaron purchase and comparing it to the recency in the rfmTable we verify our RFM table is correct.

In [21]:
(NOW - dt.datetime(2012,1,17)).days==1079

True

## Determining RFM Quartiles

In [22]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [23]:
quantiles

,recency,frequency,monetary_value
0.25,350.0,1.0,80.0
0.50,714.0,1.0,182.0
0.75,1081.0,1.0,487.0


Send quantiles to a dictionary, easier to use.

In [24]:
quantiles = quantiles.to_dict()

In [25]:
quantiles

{'frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'monetary_value': {0.25: 80.0, 0.5: 182.0, 0.75: 487.0},
 'recency': {0.25: 350.0, 0.5: 714.0, 0.75: 1081.0}}

## Creating the RFM segmentation table

In [26]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [27]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [28]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [34]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [35]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
member_name,,,,,,,
Amy Hunt,1079,1,89,3,4,3,343
Andy Reiter,348,1,5803,1,4,1,141
Anna Gayman,716,1,405,3,4,2,342
Bill Eplett,715,1,82,3,4,3,343
Brendan Sweed,1446,1,62,4,4,4,444


In [33]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [36]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort('monetary_value', ascending=False).head(5)

AttributeError: 'DataFrame' object has no attribute 'sort'